In [30]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time

In [33]:
input_path = '/home/murilo/workspace/tcc/tcc/python_model/database/x_test.csv'
output_path = '/home/murilo/workspace/tcc/tcc/python_model/database/y_test.csv'

x_test = pd.read_csv(input_path).values
y_test = pd.read_csv(output_path).values.squeeze()

In [34]:
model_keras = tf.keras.models.load_model('python_model.h5')

start_time = time.time()
predictions_keras = model_keras.predict(x_test)
end_time = time.time()

predictions_keras = predictions_keras.squeeze()

mse_keras = np.mean((y_test - predictions_keras) ** 2)
mae_keras = np.mean(np.abs(y_test - predictions_keras))

print("Keras - MSE:", mse_keras, "MAE:", mae_keras, "Tempo de Inferência:", end_time - start_time)

2/2 [==============================] - 0s 2ms/step
Keras - MSE: 0.03406967624370865 MAE: 0.13253899328129368 Tempo de Inferência: 0.05826830863952637


In [35]:
tflite_models = {
    "dynamic_range": "model_dynamic_range_quantized.tflite",
    "float16": "model_float16_quantized.tflite",
    "full_integer": "model_full_integer_quantized.tflite",
    "weights_only": "model_weights_only_quantized.tflite"
}

interpreters = {}
for key, path in tflite_models.items():
    interpreter = tf.lite.Interpreter(model_path=path)
    interpreter.allocate_tensors()
    interpreters[key] = interpreter

In [36]:
def evaluate_model(interpreter, x_test, y_test):
    input_index = interpreter.get_input_details()[0]['index']
    output_index = interpreter.get_output_details()[0]['index']
    
    # Predições TFLite
    predictions = []
    start_time = time.time()
    for test_sample in x_test:
        # Assegurar que os dados estão no formato correto e dimensionados para o modelo
        test_sample = np.expand_dims(test_sample, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_sample)
        interpreter.invoke()
        predictions.append(interpreter.get_tensor(output_index)[0][0])
    end_time = time.time()
    
    mse = np.mean((y_test - np.array(predictions)) ** 2)
    mae = np.mean(np.abs(y_test - np.array(predictions)))

    return mse, mae, end_time - start_time

In [37]:
results = {}
for key, interpreter in interpreters.items():
    mse, mae, inference_time = evaluate_model(interpreter, x_test, y_test)
    results[key] = {
        "MSE": mse,
        "MAE": mae,
        "Tempo de Inferência": inference_time
    }
    print(f"{key} - MSE: {mse}, MAE: {mae}, Tempo de Inferência: {inference_time}")

dynamic_range - MSE: 0.03421646973055114, MAE: 0.13311932799901197, Tempo de Inferência: 0.001224517822265625
float16 - MSE: 0.034069540216803626, MAE: 0.13252876251654702, Tempo de Inferência: 0.0004630088806152344


ValueError: Cannot set tensor: Got value of type FLOAT32 but expected type INT8 for input 0, name: serving_default_dense_3_input:0 